In [ ]:
import pandas as pd
import folium
import json
import requests
import numpy as np

In [ ]:
df_prefeituras_resumo_20_ = pd.read_pickle('df_prefeituras_resumo_20_pickle.pkl')

In [ ]:
df_prefeituras_resumo_20_.head()

In [ ]:
df_prefeituras_resumo_20_['CD_MUNICIPIO_IBGE'] = df_prefeituras_resumo_20_['CD_MUNICIPIO_IBGE'].apply(lambda x: str(x).replace('.0', ''))


In [ ]:
df_prefeituras_resumo_20_.head()

In [ ]:
br_cidades = 'cidades.json'

geo_json_data = json.load(open(br_cidades))

In [ ]:
# Tabela geral de municípios do IBGE
df_ibge = pd.read_excel('RELATORIO_DTB_BRASIL_MUNICIPIO.xls', encoding = 'latin1')
df_ibge['CD_MUNICIPIO_IBGE'] = df_ibge['Código Município Completo'].apply(lambda x: str(x))
df_ibge = df_ibge[['CD_MUNICIPIO_IBGE']]


In [ ]:
df_ibge.head()

In [ ]:
df_cidades_mapa = df_ibge.merge(df_prefeituras_resumo_20_, on = 'CD_MUNICIPIO_IBGE', how = 'left')#.fillna('sem_info')

def grupo_num(x):
    if x['CD_MUNICIPIO_IBGE'] == '3157336':
        y = 3
    elif x['grupo_ideologico'] == 'intervencionista_progressista':
        y = 0
    elif x['grupo_ideologico'] == 'liberal_conservador':
        y = .6
    elif x['grupo_ideologico'] == 'liberal_progressista':
        y = 1.1
    else:
        y = np.nan
    return y

df_cidades_mapa['grupo_num'] = df_cidades_mapa.apply(grupo_num, axis = 1)
df_cidades_mapa = df_cidades_mapa[['CD_MUNICIPIO_IBGE', 'grupo_num']].set_index('CD_MUNICIPIO_IBGE')['grupo_num']


In [ ]:
df_cidades_mapa.sample(10)

In [ ]:
m = folium.Map( width=1000, height=800,
    location=[-15.77972, -47.92972], 
    zoom_start=3)

folium.Choropleth(
    geo_data=geo_json_data,
    name='Perfil Ideológico',
    data=df_cidades_mapa,
#    key_on='feature.id',
    key_on='feature.properties.id',
#    fill_color='YlOrRd',    
    fill_color='Set1',
    #threshold_scale=[0, 1, 2, 3],
    line_opacity=0.1,
    legend_name='Perfil'
).add_to(m)

folium.LayerControl().add_to(m)

m